In [3]:
### Needed for google colab runtime
# import sys
# !{sys.executable} -m pip install pandas_datareader yfinance yahoo_fin pandas numpy openpyxl --upgrade --no-cache-dir

# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
import os, os.path
import errno
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import asyncio
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import numpy as np
import multiprocessing
import datetime
import time
yf.pdr_override()

In [10]:
#create directory if non-existent
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise
mkdir_p('us_stocks')

In [11]:
### Variables
tickers = pd.DataFrame()
#stock list downloaded from https://stockmarketmba.com/stocksintherussell1000.php
tickers = pd.read_excel("Russell1000list.xlsx",sheet_name=0,header=None,usecols = "A,E",skiprows = 2,names = ["ticker","sector"])
tickers["ticker"] = [item.replace(".", "-") for item in tickers["ticker"]] # Yahoo Finance uses dashes instead of dots
tickers = tickers.set_index("sector")

#combine into one string needed to download multiple files using yfinance module
tickers_string = ""
for i in tickers["ticker"]:
    tickers_string += i + " "

index_name = '^RUI' # Russell 1000
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [12]:
###  Download ticker data from yahoo finance simultaneously
results = yf.download(tickers = tickers_string,start = start_date, end = end_date, interval= '1d',threads = True,group_by="ticker")

### write to file
for ticker in tickers["ticker"]:
    df = results[ticker].reset_index()
    df.to_csv( os.path.join(os.getcwd(),'us_stocks',ticker+'.csv'), index = False, header=True )

[*********************100%***********************]  1006 of 1006 completed


In [14]:
# Index Returns
index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_return = index_df['Adj Close'].iloc[-1] / index_df['Adj Close'].iloc[0]       # not efficient: (index_df['Percent Change'] + 1).cumprod()[-1]

# Import historical data as CSV for all stocks (makes the process faster)
df_list = pd.concat(( map(pd.read_csv, [os.path.join(os.getcwd(),'us_stocks',s+'.csv') for s in tickers["ticker"] ])), keys=tickers["ticker"] )

# Find top 30% performing stocks (relative to the index)
returns_multiples = []
for ticker in tickers["ticker"]:
    # Calculating returns relative to the market (returns multiple)
    df = df_list.loc[ticker].copy()
    stock_return = df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[0]   # not efficient: ((df['Percent Change'] + 1).cumprod()).iloc[-1]
    
    returns_multiple = round((stock_return / index_return), 2)
    returns_multiples.extend([returns_multiple])
    
    print (f'Ticker: {ticker}; Returns Multiple against index: {returns_multiple}\n')

[*********************100%***********************]  1 of 1 completed
Ticker: AAPL; Returns Multiple against index: 0.99

Ticker: MSFT; Returns Multiple against index: 0.89

Ticker: GOOG; Returns Multiple against index: 0.79

Ticker: GOOGL; Returns Multiple against index: 0.79

Ticker: AMZN; Returns Multiple against index: 0.61

Ticker: BRK-B; Returns Multiple against index: 1.31

Ticker: TSLA; Returns Multiple against index: 0.61

Ticker: UNH; Returns Multiple against index: 1.41

Ticker: JNJ; Returns Multiple against index: 1.33

Ticker: XOM; Returns Multiple against index: 2.09

Ticker: NVDA; Returns Multiple against index: 0.61

Ticker: WMT; Returns Multiple against index: 1.35

Ticker: V; Returns Multiple against index: 1.22

Ticker: JPM; Returns Multiple against index: 1.02

Ticker: PG; Returns Multiple against index: 1.22

Ticker: CVX; Returns Multiple against index: 1.82

Ticker: LLY; Returns Multiple against index: 1.85

Ticker: MA; Returns Multiple against index: 1.24

Ticker:

In [15]:
# Creating dataframe of only top 40%
tickers["Returns_multiple"] =  returns_multiples
tickers["RS_Rating"] = tickers.groupby("sector").Returns_multiple.rank(pct=True) * 100
rs_df = tickers[tickers.RS_Rating >= tickers.RS_Rating.quantile(.60)].sort_values('RS_Rating',ascending=False)

In [16]:
# Checking Minervini conditions of top 40% of stocks in given list
exportList = pd.DataFrame(columns=['Stock', "Sector", "RS_Rating", "returns multiple", "Last Price","%change","50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High", "% from 52 week high"])

for i in range(len(rs_df)):
    stock = rs_df["ticker"].iloc[i]
    ret_mult = rs_df["Returns_multiple"].iloc[i]
    sector = rs_df.index[i]
    try:
        df = pd.read_csv(os.path.join(os.getcwd(),'us_stocks',f'{stock}.csv'), index_col=0) 
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(), 2)
        
        # Storing required values 
        currentClose = df["Adj Close"][-1]
        prevClose =  df["Adj Close"][-2]
        pct_chg   = (currentClose - prevClose ) * 100/ prevClose
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"]), 2)
        high_of_52week = round(max(df["High"]), 2)
        pct_from_52wkhigh = (high_of_52week - currentClose)/high_of_52week * 100
        RS_Rating = round(rs_df[rs_df['ticker']==stock].RS_Rating.tolist()[0])
        
        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price > 150 SMA and > 200 SMA
        condition_1 = currentClose > moving_average_150 > moving_average_200
        
        # Condition 2: 150 SMA and > 200 SMA
        condition_2 = moving_average_150 > moving_average_200

        # Condition 3: 200 SMA trending up for at least 1 month
        condition_3 = moving_average_200 > moving_average_200_20
        
        # Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        condition_4 = moving_average_50 > moving_average_150 > moving_average_200
           
        # Condition 5: Current Price > 50 SMA
        condition_5 = currentClose > moving_average_50
           
        # Condition 6: Current Price is at least 30% above 52 week low
        condition_6 = currentClose >= (1.3*low_of_52week)
           
        # Condition 7: Current Price is within 30% of 52 week high
        condition_7 = currentClose >= (.7*high_of_52week)

        # Condition 8: Daily change is positive 
        #
        
        # If all conditions above are true, add stock to exportList
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7):
            exportEntry = pd.DataFrame({'Stock': stock, "Sector" : sector, "RS_Rating": RS_Rating , "returns multiple" : ret_mult, "Last Price" : round(currentClose,3), "%change" : round(pct_chg,2), "50 Day MA": moving_average_50, "150 Day Ma" : moving_average_150, "200 Day MA": moving_average_200, "52 Week Low": low_of_52week, "52 week High": high_of_52week, "% from 52 week high" : round(pct_from_52wkhigh,2)},index=[0])
            exportList = pd.concat([exportList,exportEntry], ignore_index=True)
            print (stock + " made the Minervini requirements")
    except Exception as e:
        print (e)
        print(f"Could not gather data on {stock}")

exportList = exportList.sort_values(by='RS_Rating', ascending=False)
print('\n', exportList)
writer = ExcelWriter(os.path.join(os.getcwd(),"output_screener_us_long_position.xlsx"))
exportList.to_excel(writer, "Sheet1",index=False)
writer.save()

WWE made the Minervini requirements
NOC made the Minervini requirements
LW made the Minervini requirements
FSLR made the Minervini requirements
STLD made the Minervini requirements
HRB made the Minervini requirements
FICO made the Minervini requirements
FHN made the Minervini requirements
CAH made the Minervini requirements
SAIC made the Minervini requirements
ERIE made the Minervini requirements
GPC made the Minervini requirements
MCK made the Minervini requirements
ENPH made the Minervini requirements
TMUS made the Minervini requirements
RGA made the Minervini requirements
PAG made the Minervini requirements
SWCH made the Minervini requirements
CI made the Minervini requirements
HII made the Minervini requirements
WRB made the Minervini requirements
TPL made the Minervini requirements
PWR made the Minervini requirements
MRK made the Minervini requirements
PGR made the Minervini requirements
AZO made the Minervini requirements
POST made the Minervini requirements
LLY made the Minervin

/tmp/ipykernel_3801/2175041067.py:68: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [17]:
exportList

,Stock,Sector,RS_Rating,returns multiple,Last Price,%change,50 Day MA,150 Day Ma,200 Day MA,52 Week Low,52 week High,% from 52 week high
0,WWE,Communication Services,100,1.91,77.85,-1.21,76.23,69.18,66.79,46.91,81.63,4.63
2,LW,Consumer Staples,100,1.86,86.29,-1.59,83.49,76.52,72.57,49.71,88.63,2.64
3,FSLR,Information Technology,100,1.89,157.47,-5.79,144.84,108.62,100.53,59.60,173.68,9.33
4,STLD,Materials,100,2.18,108.27,1.43,91.49,80.92,80.74,50.54,109.63,1.24
5,HRB,Consumer Discretionary,100,2.14,41.52,-1.73,40.92,38.99,35.61,21.08,48.76,14.85
...,...,...,...,...,...,...,...,...,...,...,...,...
83,MNST,Consumer Staples,68,1.38,101.44,-0.71,94.70,92.17,89.46,71.78,104.65,3.07
84,CSGP,Industrials,67,1.24,80.96,-1.00,77.34,69.22,67.51,49.00,85.37,5.17
85,CTAS,Industrials,64,1.23,452.62,-1.48,421.31,403.38,401.81,343.86,463.72,2.39
86,NDAQ,Financials,63,1.19,67.01,-1.25,61.76,57.31,57.24,46.77,70.55,5.02
